In [1]:
import sys
import pandas as pd

In [2]:
df = pd.read_pickle("../data/candidates_df.pkl")

In [3]:
df[['order', 'type']].to_pickle("validation_df.pkl", protocol=3)

In [4]:
pd.read_pickle("validation_df.pkl")

,order,type
0,"[1051815, 855319, 65639, 652337]",train
1,"[73832, 561822, 102222, 561826, 80004, 177841,...",train
2,[87581],train
3,"[143731, 385579, 567, 471618]",train
4,[49241],train
...,...,...
3995,"[42767, 190556]",test
3996,[1451089],test
3997,[6109],test
3998,"[41737, 925040]",test


In [5]:
import pandas as pd
import numpy as np
from catboost import Pool, CatBoost

df = pd.read_pickle("../train_test.pkl")
pd.options.display.max_columns = 99
df.head()

,index,view,cart,type,avg_w2vec,avg_cosine,sum_w2vec,sum_cosine,len_of_w2vec,len_of_cos,len_of_pred,mean_ovr,sum_ovr,intersection_w2vec,intersection_cosine,prediction,score,target,mean_score_for_prediction,num_of_cart,num_of_views,category_1,category_2,is_popular,pred_cart_sim,pred_view_sim,prob_cart,prob_view,max_cart_sim,max_view_sim,cat1_encoded_feature,cat2_encoded_feature
0,0,"[466358, 363504, 878046, 1427398, 878047, 7355...","[1427401, 855319, 1051815, 65639, 652337]",train,0.99857,0.591565,49.928509,29.578244,50,50,100,0.795068,79.506753,0,0,1208298,0.999253,0,0.956320,2.0,3.0,"Туризм, рыбалка, охота",Рыбалка,True,0.816497,0.0,0.0,0.0,0.816497,0.0,0.004192,0.004192
1,0,"[466358, 363504, 878046, 1427398, 878047, 7355...","[1427401, 855319, 1051815, 65639, 652337]",train,0.99857,0.591565,49.928509,29.578244,50,50,100,0.795068,79.506753,0,0,389284,0.999048,0,0.915601,2.0,10.0,"Туризм, рыбалка, охота",Рыбалка,True,0.408248,0.0,0.0,0.0,0.408248,0.0,0.000000,0.000000
2,0,"[466358, 363504, 878046, 1427398, 878047, 7355...","[1427401, 855319, 1051815, 65639, 652337]",train,0.99857,0.591565,49.928509,29.578244,50,50,100,0.795068,79.506753,0,0,82739,0.999008,0,0.986127,3.0,0.0,"Туризм, рыбалка, охота",Рыбалка,True,0.333333,0.0,0.0,0.0,0.333333,0.0,0.000000,0.000000
3,0,"[466358, 363504, 878046, 1427398, 878047, 7355...","[1427401, 855319, 1051815, 65639, 652337]",train,0.99857,0.591565,49.928509,29.578244,50,50,100,0.795068,79.506753,0,0,1491380,0.998987,0,0.997073,3.0,2.0,Хобби и творчество,"Создание картин, фоторамок, открыток",True,0.000000,0.0,0.0,0.0,0.000000,0.0,0.004192,0.004192
4,0,"[466358, 363504, 878046, 1427398, 878047, 7355...","[1427401, 855319, 1051815, 65639, 652337]",train,0.99857,0.591565,49.928509,29.578244,50,50,100,0.795068,79.506753,0,0,1315601,0.998962,0,0.985984,2.0,0.0,"Туризм, рыбалка, охота",Рыбалка,True,0.408248,0.0,0.0,0.0,0.408248,0.0,0.000000,0.000000


In [6]:
train = df[
    (df.type == 'train')
].drop(['type', 'view', 'cart'], axis=1).reset_index(drop=True)
test = df[
    (df.type == 'test')
].drop(['type', 'view', 'cart'], axis=1).reset_index(drop=True)

X_train = train.drop(['index', 'target', 'prediction', 'category_1', 'category_2'], axis=1)
X_test = test.drop(['index', 'target', 'prediction', 'category_1', 'category_2'], axis=1)

y_train = train['target']
y_test = test['target']

queries_train = train['index']
queries_test = test['index']

train_prod_ids = train['prediction']
test_prod_ids = test['prediction']

In [7]:
train.head()

,index,avg_w2vec,avg_cosine,sum_w2vec,sum_cosine,len_of_w2vec,len_of_cos,len_of_pred,mean_ovr,sum_ovr,intersection_w2vec,intersection_cosine,prediction,score,target,mean_score_for_prediction,num_of_cart,num_of_views,category_1,category_2,is_popular,pred_cart_sim,pred_view_sim,prob_cart,prob_view,max_cart_sim,max_view_sim,cat1_encoded_feature,cat2_encoded_feature
0,0,0.99857,0.591565,49.928509,29.578244,50,50,100,0.795068,79.506753,0,0,1208298,0.999253,0,0.956320,2.0,3.0,"Туризм, рыбалка, охота",Рыбалка,True,0.816497,0.0,0.0,0.0,0.816497,0.0,0.004192,0.004192
1,0,0.99857,0.591565,49.928509,29.578244,50,50,100,0.795068,79.506753,0,0,389284,0.999048,0,0.915601,2.0,10.0,"Туризм, рыбалка, охота",Рыбалка,True,0.408248,0.0,0.0,0.0,0.408248,0.0,0.000000,0.000000
2,0,0.99857,0.591565,49.928509,29.578244,50,50,100,0.795068,79.506753,0,0,82739,0.999008,0,0.986127,3.0,0.0,"Туризм, рыбалка, охота",Рыбалка,True,0.333333,0.0,0.0,0.0,0.333333,0.0,0.000000,0.000000
3,0,0.99857,0.591565,49.928509,29.578244,50,50,100,0.795068,79.506753,0,0,1491380,0.998987,0,0.997073,3.0,2.0,Хобби и творчество,"Создание картин, фоторамок, открыток",True,0.000000,0.0,0.0,0.0,0.000000,0.0,0.004192,0.004192
4,0,0.99857,0.591565,49.928509,29.578244,50,50,100,0.795068,79.506753,0,0,1315601,0.998962,0,0.985984,2.0,0.0,"Туризм, рыбалка, охота",Рыбалка,True,0.408248,0.0,0.0,0.0,0.408248,0.0,0.000000,0.000000


In [8]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

train_pool = Pool(
    data=X_train,
    label=y_train,
    group_id=queries_train,
    # cat_features=['category_1', 'category_2']
)
test_pool = Pool(
    data=X_test,
    label=y_test,
    group_id=queries_test,
    # cat_features=['category_1', 'category_2']
)

default_params = {
    'iterations': 2000,
    'verbose': False,
    'task_type': 'GPU',
    'learning_rate': 0.95
}

def fit_model(loss, train_dir, add_params, train_poolt=train_pool, test_pool=test_pool):
    params = default_params
    params['loss_function'] = loss
    params['train_dir'] = train_dir
    if add_params is not None:
        params.update(add_params)
    model = CatBoost(params)
    model.fit(train_pool, eval_set=test_pool, plot=True)
    
    return model

model = fit_model('YetiRank', 'cb_logs/', {'custom_metric': ['MAP:top=10']})
    

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric MAP:top=10 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


In [9]:
model.save_model("catboost_yeti.cbm")